In [1]:
import pandas as pd
import numpy as np

from src.recommenders_lgbm import RecommenderLGBM
from src.metrics import precision_at_k

In [2]:
def precision_model(data, recommender):
    X = recommender.prepocessor.transform(data)
    X = recommender.features_gen.transform(X)
    result = X.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    result['recommended'] = result['user_id'].apply(lambda x: recommender.recommend(x))
    return result.apply(lambda row: precision_at_k(row['recommended'], row['actual']), axis=1).mean()

In [3]:
data_train = pd.read_csv('data/retail_train.csv')
data_test = pd.read_csv('data/retail_test.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

In [4]:
recommender = RecommenderLGBM(data_train, item_features, user_features)
recommender.fit()

/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/preprocessor.py:75: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  self.avg_chek = data.groupby('user_id')['sales_value', 'basket_id']\
/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/preprocessor.py:83: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  .groupby(['user_id', 'department'])['quantity', 'sales_value']\
/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/preprocessor.py:150: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  .groupby('department')['item_id', 'avg_item_price']\
/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/preprocessor.py:170: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) wi

In [5]:
result = {}

result['train'] = round(precision_model(recommender.data_train_lvl_2, recommender), 3)
result['valid'] = round(precision_model(recommender.data_val_lvl_2, recommender), 3)
result['test'] = round(precision_model(data_test, recommender), 3)

pd.DataFrame(pd.Series(result), columns=['precision_at_5'])

/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/recommenders_lgbm.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['preds'] = self.model.predict_proba(X)[:, 1]


,precision_at_5
train,1.000
valid,0.979
test,0.983


In [6]:
# save recommendations to recommendations.csv
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result['recommendations'] = result['user_id'].apply(lambda x: recommender.recommend(x))
result[['user_id', 'recommendations']].to_csv('recommendations.csv', index=False)

/home/sergey/Документы/11. рекомендательные системы/webinar_8/src/recommenders_lgbm.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['preds'] = self.model.predict_proba(X)[:, 1]
